# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [41]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [111]:
xcities_df = pd.read_csv('../Resources/cities.csv')
cities_df = xcities_df.drop(columns=['Unnamed: 0'])
cities_df

,city,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,qaanaaq,77.4840,-69.3632,40.33,75.0,4,3.13,GL,1627007456
1,isla mujeres,21.2311,-86.7310,84.25,85.0,0,8.39,MX,1627007456
2,puerto ayora,-0.7393,-90.3518,72.55,87.0,97,4.00,EC,1627007457
3,hasaki,35.7333,140.8333,90.28,67.0,80,8.72,JP,1627007457
4,provideniya,64.3833,-173.3000,46.51,76.0,75,11.18,RU,1627007457
...,...,...,...,...,...,...,...,...,...
548,arica,-18.4750,-70.3042,59.23,82.0,90,5.75,CL,1627007607
549,souillac,-20.5167,57.5167,70.18,78.0,20,11.50,MU,1627007607
550,katherine,-14.4667,132.2667,86.32,30.0,33,5.75,AU,1627007608
551,tanggu,39.0211,117.6469,85.96,62.0,100,1.52,CN,1627007608


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [112]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

locations = cities_df[['Latitude', 'Longitude']]
humidity = cities_df['Humidity']

In [129]:
# Plot Heatmap
figure_layout = {
    'width': '750px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

paris_coordinates = (48.8566, 2.3522)
fig = gmaps.figure(layout=figure_layout, center = paris_coordinates, zoom_level=1.6)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [103]:
exp_hotel_df = cities_df[(cities_df['Max Temp'] < 90) & (cities_df['Max Temp'] > 50) & (cities_df['Cloudiness'] < 20) & (cities_df['Humidity'] < 30)].reset_index()
hotel_df = exp_hotel_df.drop(columns=['index'])

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [104]:
hotel_df['Hotel Name'] = ""
hotel_df

,city,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,erenhot,43.6475,111.9767,86.56,29.0,5,9.31,CN,1627007464,
1,adrar,20.5022,-10.0711,85.66,17.0,5,9.69,MR,1627007483,
2,airai,-8.9266,125.4092,76.12,23.0,7,3.49,TL,1627007493,
3,bend,44.0582,-121.3153,81.68,22.0,1,8.99,US,1627006654,
4,temir,49.1414,57.1300,85.01,25.0,3,11.88,KZ,1627007501,
5,sandpoint,48.2766,-116.5533,79.56,26.0,1,1.01,US,1627007293,
6,gazanjyk,39.2446,55.5154,87.37,16.0,6,4.56,TM,1627007535,
7,karratha,-20.7377,116.8463,80.49,22.0,7,12.77,AU,1627007578,
8,edson,53.5835,-116.4356,64.54,29.0,1,11.50,CA,1627007580,
9,soe,-9.8607,124.2840,83.05,23.0,16,6.60,ID,1627007605,


In [105]:
# find the closest hotel to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    city = row["city"]
    
    #assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    try:
        print(f"Closest hotel for {city} is {response['results'][0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        print(response['results'][0]['name'])
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results for Index 0: erenhot.
Closest hotel for erenhot is Erlian Pacific International Hotel.
Erlian Pacific International Hotel
Retrieving Results for Index 1: adrar.
Missing field/result... skipping.
Retrieving Results for Index 2: airai.
Missing field/result... skipping.
Retrieving Results for Index 3: bend.
Closest hotel for bend is Bunk+Brew Historic Lucas House.
Bunk+Brew Historic Lucas House
Retrieving Results for Index 4: temir.
Missing field/result... skipping.
Retrieving Results for Index 5: sandpoint.
Closest hotel for sandpoint is Best Western Plus Ponderay Mountain Lodge.
Best Western Plus Ponderay Mountain Lodge
Retrieving Results for Index 6: gazanjyk.
Closest hotel for gazanjyk is Railway motel.
Railway motel
Retrieving Results for Index 7: karratha.
Closest hotel for karratha is Best Western Plus The Ranges Karratha.
Best Western Plus The Ranges Karratha
Retrieving Results for Index 8: edson.
Closest hotel for edson is Holiday Inn Express & Suites Edson, an

In [106]:
hotel_df

,city,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,erenhot,43.6475,111.9767,86.56,29.0,5,9.31,CN,1627007464,Erlian Pacific International Hotel
1,adrar,20.5022,-10.0711,85.66,17.0,5,9.69,MR,1627007483,
2,airai,-8.9266,125.4092,76.12,23.0,7,3.49,TL,1627007493,
3,bend,44.0582,-121.3153,81.68,22.0,1,8.99,US,1627006654,Bunk+Brew Historic Lucas House
4,temir,49.1414,57.1300,85.01,25.0,3,11.88,KZ,1627007501,
5,sandpoint,48.2766,-116.5533,79.56,26.0,1,1.01,US,1627007293,Best Western Plus Ponderay Mountain Lodge
6,gazanjyk,39.2446,55.5154,87.37,16.0,6,4.56,TM,1627007535,Railway motel
7,karratha,-20.7377,116.8463,80.49,22.0,7,12.77,AU,1627007578,Best Western Plus The Ranges Karratha
8,edson,53.5835,-116.4356,64.54,29.0,1,11.50,CA,1627007580,"Holiday Inn Express & Suites Edson, an IHG Hotel"
9,soe,-9.8607,124.2840,83.05,23.0,16,6.60,ID,1627007605,Dena Hotel


In [107]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>city</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Latitude", "Longitude"]]

In [131]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[['Latitude', 'Longitude']]
hotel_humidity = hotel_df['Humidity']

# Create a marker_layer using the poverty list to fill the info box

fig = gmaps.figure(layout=figure_layout, center = paris_coordinates, zoom_level=1.6)

heat_layer = gmaps.heatmap_layer(hotel_locations, weights=hotel_humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

markers = gmaps.marker_layer(marker_locations,
    [info_box_template.format(**row) for index, row in hotel_df.iterrows()])
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…